# This is a notebook for querying the EMODnet Chemistry instance.

- You can run each cell individually by pressing "shift + enter".
- Please run the first few cells in order for the input widgets to appear.
- For more information, questions, bugs, please contact us on Slack:
  - https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token that you should receive via Blue-Cloud between the '' " in the cell below.


In [1]:
Token = ''

#### Importing the required packages.


In [2]:
import requests
import json
import xarray as xr
import datetime
from ipywidgets import interact, widgets, interactive
from IPython.display import Javascript, display
import pandas as pd
import os 

#### This retrieves the available parameters from the footprint.


In [3]:
if not os.path.exists('./Footprint'):
    os.makedirs('Footprint')

responseinfo = requests.get("https://beacon-emod-chem.maris.nl/api/datasets/footprint", headers = {'Authorization' : f'Bearer {Token}'}) 
open('./Footprint/Footprint_emod_chem.json', 'wb').write(responseinfo.content)
with open('./Footprint/Footprint_emod_chem.json', 'r') as f:
    data = json.load(f)

pd.set_option('display.max_rows', 500)
attributes = pd.DataFrame.from_dict(data['unique_column_attributes'], orient='index')
attributes.reset_index(level=0, inplace=True)
attributes = attributes.sort_values('index', ascending=True, key=lambda col: col.str.lower())
params = attributes['index'].to_numpy()

#### In the widgets below, you should select your preferred filters.


In [4]:
def f(parameter, minlon, maxlon, minlat, maxlat, mindepth, maxdepth, mintemp, maxtemp):
    return parameter, minlon, maxlon, minlat, maxlat, mindepth, maxdepth, mintemp, maxtemp

w = interactive(f, parameter=params, minlon=(-180, 180, 1), maxlon=(-180, 180, 1), minlat=(-90, 90, 1), maxlat=(-90, 90, 1), mindepth=widgets.IntSlider(min=0, max=5000, value=0), maxdepth=widgets.IntSlider(
    min=0, max=5000, value=10), mintemp=widgets.DatePicker(description='Start date:', value=datetime.date(2000, 1, 1)), maxtemp=widgets.DatePicker(description='End date:', value=datetime.date(2024, 1, 1)))

display(w)

interactive(children=(Dropdown(description='parameter', options=('Access_ordering_of_data', 'Access_restrictio…

#### Storing the chosen filters.


In [9]:
parameter = w.kwargs['parameter']
mintemp = w.kwargs['mintemp']
maxtemp = w.kwargs['maxtemp']
minlon = w.kwargs['minlon']
maxlon = w.kwargs['maxlon']
minlat = w.kwargs['minlat']
maxlat = w.kwargs['maxlat']
mindepth = w.kwargs['mindepth']
maxdepth = w.kwargs['maxdepth']

#### The fields below are fixed for the EMODnet Chemistry instance.


In [10]:
unit = attributes.loc[attributes['index'] == parameter]['units'].iloc[0]
dateref = datetime.date(1921, 1, 1)  # Days since 1921
mintemporal = (mintemp - dateref).days  # Days since 1921
maxtemporal = (maxtemp - dateref).days  # Days since 1921
regionname = f'[{minlat},{minlon}],[{maxlat},{maxlon}]'

#### Query body based on your input fields, you can add other "query_parameters" and "filters" to suit your needs.

- For more query examples, you can take a look at https://maris-development.github.io/beacon/available-nodes/blue-cloud-2026/emodnet-chemistry.html.


In [11]:
def query(parameter, mintemporal, maxtemporal, mindepth, maxdepth, minlon, maxlon, minlat, maxlat):
    body = {
        "query_parameters": [
            {
                "column_name": "ITS_90_water_temperature",
                "alias": "Temperature [celsius]"
            },
            {
                "column_name": "ITS_90_water_temperature_qc",
                "alias": "Temperature_qc"
            },
            {
                "column_name": "date_time",
                "alias": "TIME"
            },
            {
                "function": "time::const_convert_datetime_to_iso",
                "epoch": "1921-01-01T00:00:00Z",
                "unit": "days",
                "input": {
                    "column_name": "date_time"
                },
                "alias": "IsoTime"
            },
            {
                "column_name": "Depth",
                "alias": "Depth [meter]"
            },
            {
                "column_name": "Depth_qc",
                "alias": "Depth_qc"
            },
            {
                "column_name": "longitude",
                "alias": "LONGITUDE"
            },
            {
                "column_name": "latitude",
                "alias": "LATITUDE"
            },
            {
                "column_name": "EDMO_code",
                "alias": "EDMO_code"
            },
        ],
        "filters": [
            {
                "for_query_parameter": "TIME",
                "min": mintemporal,
                "max": maxtemporal
            },
            {
                "for_query_parameter": "Depth [meter]",
                "min": mindepth,
                "max": maxdepth
            },
            {
                "for_query_parameter": "LONGITUDE",
                "min": minlon,
                "max": maxlon
            },
            {
                "for_query_parameter": "LATITUDE",
                "min": minlat,
                "max": maxlat
            }
        ],
        "output": {
            "format": {
                "odv": {
                    "longitude_column": "LONGITUDE",
                    "latitude_column": "LATITUDE",
                    "timestamp_column": {
                        "data_column_name": "IsoTime",
                        "comment": ""
                    },
                    "depth_column": {
                        "data_column_name": "Depth [meter]",
                        "comment": "",
                        "qf_column": "Depth_qc"
                    },
                    "data_columns": [
                        {
                            "data_column_name": "Temperature [celsius]",
                            "comment": "",
                            "qf_column": "Temperature_qc"
                        }
                    ],
                    "metadata_columns": [
                        "EDMO_code"
                    ],
                    "skip_odv_script": False,
                    "qf_schema": "SEADATANET"
                }
            }
        }}
    return body


query_body = query(parameter, mintemporal, maxtemporal, mindepth,
              maxdepth, minlon, maxlon, minlat, maxlat)

#### This is the post request that is sent to Beacon with the above specified body.


In [12]:
response = requests.post("https://beacon-emod-chem.maris.nl/api/query", json.dumps(query_body), headers={
    'Authorization': f'Bearer {Token}',
    'Content-type': 'application/json'})

if response.status_code == 204:
    print('No data has been found for your query, please update your input fields above and run the notebook again.')
elif response.status_code != 200:
    print(response.text)

#### This will create a ODV file in your directory with the name based on your filters.


In [13]:
open(f'EMODNET_CHEM_{parameter}_{regionname}_{mintemp.strftime("%Y-%m-%d")}-{maxtemp.strftime("%Y-%m-%d")}_[{mindepth}-{maxdepth}m].zip', 'wb').write(response.content);